In [50]:
#Grille jeux + alternance jetons/joueurs + sauvegarde/charger partie Maya : 
# + check gagnant/match nul + nouvelle partie + quelques modifs Kinga

#biblio nécessaire
import tkinter as tk
import numpy as np
from PIL import Image, ImageTk
from tkinter import filedialog
import json
import random

historique_coups = []

#https://docs.python.org/3/library/tkinter.html

# Taille de la grille de jeux
lignes = 6
colonnes = 7




# Couleurs utilisées : 
#Alors pour l'instant j'ai juste mis les couleur du jeux de base, a voir par la suite si on modifie

bleu = (0, 0, 255)  
rouge = (255, 0, 0)  
jaune = (255, 255, 0)
noir = (0,0,0)


# Création fenêtre
fenetre_grillejeux = tk.Tk()

#Prend taille écran en compte perso
ecran_largeur = fenetre_grillejeux.winfo_screenwidth() #https://sites.google.com/site/pythonpasapas/modules/tkinter/tkinter-methodes/tkinter-winfo_screenwidth
ecran_hauteur = fenetre_grillejeux.winfo_screenheight()

# donc ça devrait adapter 
taille_case = min(ecran_largeur // (colonnes + 2), ecran_hauteur // (lignes + 4))
rayon = int(taille_case / 2 - 5)


fenetre_grillejeux.geometry(f"{ecran_largeur}x{ecran_hauteur}") 
fenetre_grillejeux.title("Grille de jeux")
fenetre_grillejeux.state('zoomed')


# Compteur de victoires
victoires_joueur1 = 0
victoires_joueur2 = 0

manche = random.randint(0, 1)

# Label message gagnant dans le canvas
label_resultat = tk.Label(fenetre_grillejeux, text="Que le meilleur gagne ! 🎯", font=("Impact", 50))
label_resultat.pack(pady=(30, 0))

prochain_joueur = "Rouge" if (cmpt + manche) % 2 == 0 else "Jaune"
couleur_texte = "red" if prochain_joueur == "Rouge" else "#FFD700"
label_joueur = tk.Label(fenetre_grillejeux, text = f"{prochain_joueur} commence !", font=("Impact", 20), fg = couleur_texte)
label_joueur.pack()

#frames pour séparer canvas et boutons normalement aide si boutons apparaissent pas sur votre écran (normalement)
frame_canvas = tk.Frame(fenetre_grillejeux)
frame_canvas.pack()

frame_boutons = tk.Frame(fenetre_grillejeux)
frame_boutons.pack(pady=10)

canvas = tk.Canvas(frame_canvas, width=colonnes * taille_case, height=(lignes + 1) * taille_case)
canvas.pack()


# Création grille de base donc vide
def creation_grille():
    grille = np.zeros((lignes, colonnes))  #soit matrice 6x7
    return grille

grille = creation_grille() 



#Afficher messages quand réalise action boutons (à voir et rendre plus propre)
message_label = tk.Label(fenetre_grillejeux, text="", font=("Arial", 14), fg="green")
message_label.pack(pady=10)



# Fonction pour tracer grille et emplacements jetons
def tracer_grille(grille):
    canvas.delete("all") 
    # Base de la grille
    for i in range(colonnes):
        for j in range(lignes):
            canvas.create_rectangle(i*taille_case, j*taille_case + taille_case, (i + 1)*taille_case, (j + 1)*taille_case + taille_case, fill="blue", outline="black")
            canvas.create_oval(i*taille_case + 10, j*taille_case + taille_case + 10, (i + 1)*taille_case - 10, (j + 1)*taille_case + taille_case - 10, fill="black")
    # Places jetons
    for i in range(colonnes):
        for j in range(lignes):
            if grille[j][i] == 1:  # pour quand alterne J1
                canvas.create_oval(i*taille_case + 10, (lignes -j - 1)*taille_case + taille_case + 10, (i + 1)*taille_case - 10, (lignes -j)*taille_case + taille_case - 10, fill="red")
            elif grille[j][i] == 2:  # pour quand alterne J2
                canvas.create_oval(i* taille_case + 10, (lignes -j - 1)*taille_case + taille_case + 10, (i + 1)*taille_case - 10, (lignes-j)*taille_case + taille_case - 10, fill="yellow")

#décalage pour l'instant en fonction de la taille grille mais a rendre modifiable

# Alors j'ai pensé un peu au placement des jetons par clic, c'est pas encore ça + faut alterner les joueurs mieux mais l'idée est là
# doc thinker event : https://docs.python.org/3/library/tkinter.html#bindings-and-events


# Kinga : J'ai modifié cette fonction comme ca si y a une colonne remplie il se passe rien qd on appuie
def ajouter_jeton(grille, col, J):
    for i in range(0, lignes):
        if grille[i][col] == 0:
            grille[i][col] = J
            return True             # Jeton placé
    return False                    # Si colonne remplie


def alternance_joueur(event, grille, taille_case, colonnes, cmpt):
    colonne_clique = event.x // taille_case  #position x de la souris donc la colonne choisie
    if colonne_clique >= colonnes:
        return grille, cmpt

    if (cmpt + manche) % 2 == 0: 
        J = 1 #là sera rouge
    else:  
        J = 2 #là sera jaune 

    coup_valide = ajouter_jeton(grille, colonne_clique, J)      # Kinga : idem ici
    if coup_valide == False :
        return grille, cmpt
    
    tracer_grille(grille)
    cmpt += 1
    return grille, cmpt


#pour gérer le nbr de clic/alternance
# Kinga : ajout des coups gagnant/ match nul
def clic_canvas(event):
    global grille, cmpt

        # On joue le coup
    grille, cmpt = alternance_joueur(event, grille, taille_case, colonnes, cmpt)

        # Puis on vérifie s'il y a un gagnant
    gagnant = check_gagnant(grille)

    if gagnant != 0:
        couleur = "Rouge" if gagnant == 1 else "Jaune"
        if gagnant == 1:
            global victoires_joueur1
            victoires_joueur1 += 1
            label_resultat.config(text=f"Rouge a gagné ! 🎉", fg="red")
        else:
            global victoires_joueur2
            victoires_joueur2 += 1
            label_resultat.config(text=f"Jaune a gagné ! 🎉", fg="#FFD700")
        
        canvas.unbind("<Button-1>")         # Désactive les clics
        label_joueur.config(text="")


    # Vérifier si la grille est pleine et qu'il n'y a pas de gagnant -> Match nul
    elif np.all(grille != 0):
        label_resultat.config(text="Match nul ! 😅", fg = "black")
        canvas.unbind("<Button-1>")         # Désactive les clics


    if gagnant == 0 and not np.all(grille != 0):
        prochain_joueur = "Rouge" if (cmpt + manche) % 2 == 0 else "Jaune"
        couleur_texte = "red" if prochain_joueur == "Rouge" else "#FFD700"
        label_resultat.config(text=f"À {prochain_joueur} de jouer !", fg=couleur_texte)
    
    label_joueur.config(text = "")


cmpt = 0 #début compteur

canvas.bind("<Button-1>", clic_canvas)

tracer_grille(grille)

#Remettre a zero la grille à chaque ouverture de la fenetre :

def reset_grille():
    global grille, cmpt, historique_coups, manche
    manche += 1
    grille = creation_grille()  
    cmpt = 0  # clic 0
    historique_coups.clear()  #pour éviter historique garde ancien coups
    tracer_grille(grille)
    message_label.config(text="Grille bien réinitialisée", fg="orange")
    fenetre_grillejeux.after(3000, lambda: message_label.config(text="")) #j'ai mis 3s pour pas que ça reste infiniment 
    canvas.bind("<Button-1>", clic_canvas)      # On réactive les clics
    label_resultat.config(                      # Efface le message de victoire
    text=f"Rouge - {victoires_joueur1}  ||  Jaune - {victoires_joueur2}",
        fg="black"
    )
    prochain_joueur = "Rouge" if (cmpt + manche) % 2 == 0 else "Jaune"
    couleur_texte = "red" if prochain_joueur == "Rouge" else "#FFD700"
    label_joueur.config(text = f"{prochain_joueur} commence !", fg = couleur_texte)



#fonction sauvegarder/charger pour l'instant

def sauvegarder_partie():
    nom_fichier = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("Fichier JSON", "*.json")]) #comme ça on écrit ce qu'on veut comme nom
    if not nom_fichier:
        return
    donnees = {
        "grille": grille.tolist(),
        "tour": cmpt,
        "historique": historique_coups
    }
    with open(nom_fichier, "w") as fichier:
        json.dump(donnees, fichier)
    message_label.config(text=f"Partie sauvegardée sous {nom_fichier}", fg="green")
    fenetre_grillejeux.after(3000, lambda: message_label.config(text=""))


def charger_partie():
    global grille, cmpt, historique_coups
    nom_fichier = filedialog.askopenfilename(defaultextension=".json", filetypes=[("Fichier JSON", "*.json")])
    if not nom_fichier:
        return
    with open(nom_fichier, "r") as fichier:
        donnees = json.load(fichier)
    grille = np.array(donnees["grille"])
    cmpt = donnees["tour"]
    historique_coups = donnees["historique"]
    tracer_grille(grille)
    message_label.config(text=f"Partie chargée à partir de {nom_fichier}", fg="blue")
    fenetre_grillejeux.after(3000, lambda: message_label.config(text=""))

def nouvelle_partie():
    global grille, cmpt, victoires_joueur1, victoires_joueur2, manche
    manche = random.randint(0, 1)
    grille = creation_grille()
    cmpt = 0
    victoires_joueur1 = 0
    victoires_joueur2 = 0
    tracer_grille(grille)
    canvas.bind("<Button-1>", clic_canvas)
    label_resultat.config(text="Que le meilleur gagne ! 🎯", fg="black")
    prochain_joueur = "Rouge" if (cmpt + manche) % 2 == 0 else "Jaune"
    couleur_texte = "red" if prochain_joueur == "Rouge" else "#FFD700"
    label_joueur.config(text = f"{prochain_joueur} commence !", fg = couleur_texte)

def check_gagnant(grille):
    lignes, colonnes = grille.shape

    for ligne in range(lignes):
        for col in range(colonnes):
            joueur = grille[ligne][col]
            if joueur == 0:
                continue

            # Horizontal
            if col + 3 < colonnes and all(grille[ligne][col + i] == joueur for i in range(4)):
                return joueur

            # Vertical
            if ligne + 3 < lignes and all(grille[ligne + i][col] == joueur for i in range(4)):
                return joueur

            # Diagonale descendante
            if ligne + 3 < lignes and col + 3 < colonnes and all(grille[ligne + i][col + i] == joueur for i in range(4)):
                return joueur

            # Diagonale montante
            if ligne - 3 >= 0 and col + 3 < colonnes and all(grille[ligne - i][col + i] == joueur for i in range(4)):
                return joueur

    return 0  # pas de gagnant


#Tous les bontons a afficher : 
bouton_reset = tk.Button(frame_boutons, text="Nettoyer la grille", command=reset_grille)
bouton_reset.pack(side=tk.LEFT, padx=10)

bouton_nvl_partie = tk.Button(frame_boutons, text = "Recommencer", command = nouvelle_partie)
bouton_nvl_partie.pack(side=tk.LEFT, padx=10)

bouton_sauvegarder = tk.Button(frame_boutons, text="Sauvegarder", command=sauvegarder_partie)
bouton_sauvegarder.pack(side=tk.LEFT, padx=10)

bouton_charger = tk.Button(frame_boutons, text="Charger", command=charger_partie)
bouton_charger.pack(side=tk.LEFT, padx=10)


fenetre_grillejeux.mainloop()


###########################################################################################################

